# NBN FWA Data Processing

2017 Hermes search containing licences from all NBN licence holdings.

> Note: TPA refers to calculated total power into antenna and power refers to reported power values.

## Calculating Total Power into Antenna (TPA)

TPA = Total Power into Antenna

$G_a$ = Antenna Gain

$TPA_{dBW} = EIRP_{dBW} - G_{a(dBi)}$

$TPA_{dBW} = EIRP_{dBm} - 30 - G_{a(dBi)}$

Spectral Density $ = EIRP_{dBm} + 10log_{10}(\frac{1}{BW})$

### Unit Conversion Formulas

$P_{dbW} = 10 \times log_{10}(P_W)$

$P_W = 10^{\frac{P_{dBW}}{10}}$

$P_{dBm} = P_{dBW} + 30$

In [2]:
import pandas as pd
import numpy as np

nbn = pd.read_csv('Data/NBNFWA2017.csv')

# Remove search irrelevant data
nbn = nbn[[
    'licence_number',
    'bandwidth',
    'freq_lower',
    'freq_upper',
    'service_type',
    'height',
    'latitude',
    'longitude',
    'site_name',
    'state',
    'client',
    'tx_rx',
    'power_watts',
    'eirp_watts',
    'antenna_gain'
]]

# Filter out Rx entries
nbn = nbn[nbn['tx_rx'] == 'T']

# Hz to MHz
nbn['bandwidth'] = (nbn['bandwidth'] / 1000000).round(3)
nbn['freq_upper'] = (nbn['freq_upper'] / 1000000).round(7)
nbn['freq_lower'] = (nbn['freq_lower'] / 1000000).round(7)
nbn.rename(columns={
    'bandwidth':'BW (MHz)',
    'freq_upper' : 'Freq_Upper',
    'freq_lower' : 'Freq_Lower',
    'eirp_watts' : 'EIRP',
    'height' : 'Antenna_Ht',
    'antenna_gain' : 'Antenna_Gain'
    }, inplace=True)

# Convert W to dBm
nbn['EIRP_dBm'] = 0
nbn = nbn.astype({'EIRP' : 'float','EIRP_dBm' : 'float'})
nbn = nbn.replace({0:np.nan})
nbn['EIRP_dBm'] = 10 * np.log10(1000 * nbn['EIRP'])
nbn['EIRP_dBm/MHz'] = nbn['EIRP_dBm'] + 10 * np.log10(1 / nbn['BW (MHz)'])

nbn['power_dBm'] = 0
nbn = nbn.astype({'power_watts' : 'float','power_dBm' : 'float'})
nbn = nbn.replace({0:np.nan})
nbn['power_dBm'] = 10 * np.log10(1000 * nbn['power_watts'])
nbn['power_dBm/MHz'] = nbn['power_dBm'] + 10 * np.log10(1 / nbn['BW (MHz)'])

# Total Power into Antenna
nbn['TPA_dBW'] = (nbn['EIRP_dBm'] - 30) - nbn['Antenna_Gain']
nbn['TPA_W'] = np.power(10, (nbn['TPA_dBW'] / 10))
nbn['TPA_dBm/MHz'] = nbn['TPA_dBW'] + 10 * np.log10(1 / nbn['BW (MHz)'])
nbn['MHz_dBm'] = 10 * np.log10(1 / nbn['BW (MHz)'])

# Save to xlsx
nbn.reset_index(drop=True, inplace=True)
nbn.to_excel('Data/NBN/nbn.xlsx', index=False)
nbn

,licence_number,BW (MHz),Freq_Lower,Freq_Upper,service_type,Antenna_Ht,latitude,longitude,site_name,state,...,EIRP,Antenna_Gain,EIRP_dBm,EIRP_dBm/MHz,power_dBm,power_dBm/MHz,TPA_dBW,TPA_W,TPA_dBm/MHz,MHz_dBm
0,9867483,19.9,3445.05,3464.95,Spectrum,40.0,-37.859231,140.757389,NBN Co Site 21444 Riddoch Highway MOORAK,SA,...,8592.509,17.5,69.341200,56.352669,52.0412,39.052669,21.841200,152.798818,8.852669,-12.988531
1,9867483,19.9,3445.05,3464.95,Spectrum,22.5,-37.055978,146.088507,Telstra Exchange 45 Hunter St MANSFIELD,VIC,...,8592.509,17.5,69.341200,56.352669,52.0412,39.052669,21.841200,152.798818,8.852669,-12.988531
2,9867483,19.9,3445.05,3464.95,Spectrum,40.0,-35.959135,146.889159,NBN Co Site Dight St JINDERA,NSW,...,8592.509,17.5,69.341200,56.352669,52.0412,39.052669,21.841200,152.798818,8.852669,-12.988531
3,9867483,19.9,3445.05,3464.95,Spectrum,45.0,-38.278652,142.473039,NBN Co Site 476 Ibbs Lane MAILORS FLAT,VIC,...,8592.509,17.5,69.341200,56.352669,52.0412,39.052669,21.841200,152.798818,8.852669,-12.988531
4,9867483,19.9,3550.05,3569.95,Spectrum,40.0,-36.229948,145.560267,NBN Co Site 238-240 Hickey Road KATANDRA WEST,VIC,...,8592.509,17.5,69.341200,56.352669,52.0412,39.052669,21.841200,152.798818,8.852669,-12.988531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,9997701/60,20.0,3517.50,3537.50,PTS,40.0,-33.623880,150.667570,NBN Co Site 73 Mountain Avenue YARRAMUNDI,NSW,...,8019.000,17.5,69.041202,56.030902,52.0412,39.030900,21.541202,142.600226,8.530902,-13.010300
3246,9997701/60,20.0,3497.50,3517.50,PTS,40.0,-32.618580,151.718900,NBN Co Site 18 Langlands Road GLEN OAK,NSW,...,8019.000,17.5,69.041202,56.030902,52.0412,39.030900,21.541202,142.600226,8.530902,-13.010300
3247,9997701/60,20.0,3517.50,3537.50,PTS,40.0,-32.618580,151.718900,NBN Co Site 18 Langlands Road GLEN OAK,NSW,...,8019.000,17.5,69.041202,56.030902,52.0412,39.030900,21.541202,142.600226,8.530902,-13.010300
3248,9997701/60,20.0,3400.00,3420.00,PTS,49.7,-32.881283,151.380654,NBN Co Site 96 Abernethy Street KITCHENER,NSW,...,8019.000,17.5,69.041202,56.030902,52.0412,39.030900,21.541202,142.600226,8.530902,-13.010300


In [3]:
# Bandwidth Table
BWstats = pd.DataFrame(nbn['BW (MHz)'].value_counts(sort=False))
BWstats['Percentage (%)'] = ((BWstats['BW (MHz)'] /  3250) * 100).round(2)
BWstats.rename(columns={'BW (MHz)' : 'Bandwidth (MHz)'}, inplace=True)
BWstats.to_csv('Data/NBN/BWstats.csv', index=True)
BWstats

,Bandwidth (MHz),Percentage (%)
19.9,952,29.29
20.0,2298,70.71


In [15]:
# Bandwidth Distribution
bins = [0, 5, 10, 15, 20]
BWdist = pd.DataFrame(nbn['BW (MHz)'].value_counts(bins=bins, sort=False))
BWdist['Percentage (%)'] = ((BWdist['BW (MHz)'] /  3250) * 100).round(2)
BWdist.rename(columns={'BW (MHz)' : 'Bandwidth (MHz)'}, inplace=True)
BWdist.to_csv('Data/NBN/BWdist.csv', index=True)
BWdist

,Bandwidth (MHz),Percentage (%)
"(-0.001, 5.0]",0,0.0
"(5.0, 10.0]",0,0.0
"(10.0, 15.0]",0,0.0
"(15.0, 20.0]",3250,100.0


In [16]:
# Power Table
power = pd.DataFrame(nbn['power_watts'].value_counts(sort=False))
power['Percentage (%)'] = ((power['power_watts'] /  3250) * 100).round(2)
power.rename(columns={'power_watts' : 'Power (W)'}, inplace=True)
power.to_csv('Data/NBN/powerTable.csv', index=True)
power

,Power (W),Percentage (%)
160.0,3243,99.78
80.0,7,0.22


In [17]:
# Power Table (dBm/MHz)
powerDist = pd.DataFrame(nbn['power_dBm/MHz'].value_counts( sort=True))
powerDist['Percentage (%)'] = ((powerDist['power_dBm/MHz'] /  3250) * 100).round(2)
powerDist.rename(columns={'power_dBm/MHz' : 'Power (dBm/MHz)'}, inplace=True)
powerDist.to_csv('Data/NBN/powerDist_dBmMHz.csv', index=True)
powerDist


,Power (dBm/MHz),Percentage (%)
39.030900,2298,70.71
39.052669,945,29.08
36.042369,7,0.22


In [19]:
# TPA Table
tpa = pd.DataFrame(nbn['TPA_W'].value_counts(sort=True))
tpa['Percentage (%)'] = ((tpa['TPA_W'] /  3250) * 100).round(2)
tpa.rename(columns={'TPA_W' : 'Total Power into Antenna (W)'}, inplace=True)
tpa.to_csv('Data/NBN/tpaTable.csv', index=True)
tpa

,Total Power into Antenna (W),Percentage (%)
142.600226,1728,53.17
152.798818,637,19.60
142.600279,569,17.51
139.354155,251,7.72
152.807550,59,1.82
76.399329,3,0.09
139.351414,2,0.06
87.926587,1,0.03


In [22]:
# TPA Distribution (dBm/MHz)
TPAdist = pd.DataFrame(nbn['TPA_dBm/MHz'].value_counts( sort=True))
TPAdist['Percentage (%)'] = ((TPAdist['TPA_dBm/MHz'] /  3250) * 100).round(2)
TPAdist.rename(columns={'TPA_dBm/MHz' : 'Total Power into Antenna (dBm/MHz)'}, inplace=True)
TPAdist.to_csv('Data/NBN/TPAdist_dBmMHz.csv', index=True)
TPAdist

,Total Power into Antenna (dBm/MHz),Percentage (%)
8.530902,1728,53.17
8.852669,637,19.60
8.530904,569,17.51
8.452668,251,7.72
8.852917,59,1.82
5.842365,3,0.09
8.452583,2,0.06
6.430902,1,0.03


In [23]:
# Antenna Gains
AntennaG = pd.DataFrame(nbn['Antenna_Gain'].value_counts( sort=False))
AntennaG['Percentage (%)'] = ((AntennaG['Antenna_Gain'] /  3250) * 100).round(2)
AntennaG.rename(columns={'Antenna_Gain' : 'Antenna Gain (dBi)'}, inplace=True)
AntennaG.to_csv('Data/NBN/AntennaGain.csv', index=True)
AntennaG

,Antenna Gain (dBi),Percentage (%)
17.5,2427,74.68
19.6,823,25.32


In [24]:
# EIRP 
EIRPs = pd.DataFrame(nbn['EIRP'].value_counts( sort=True))
EIRPs['Percentage (%)'] = ((EIRPs['EIRP'] /  3250) * 100).round(2)
EIRPs.rename(columns={'EIRP' : 'EIRP (W)'}, inplace=True)
EIRPs.to_csv('Data/NBN/EIRPs.csv', index=True)
EIRPs

,EIRP (W),Percentage (%)
8019.000,1729,53.20
8592.509,637,19.60
13005.300,569,17.51
12709.250,251,7.72
8593.000,59,1.82
4296.250,3,0.09
12709.000,2,0.06


In [25]:
# EIRP dBm
EIRPs_dBm = pd.DataFrame(nbn['EIRP_dBm'].value_counts( sort=True))
EIRPs_dBm['Percentage (%)'] = ((EIRPs_dBm['EIRP_dBm'] /  3250) * 100).round(2)
EIRPs_dBm.rename(columns={'EIRP_dBm' : 'EIRP (dBm)'}, inplace=True)
EIRPs_dBm.to_csv('Data/NBN/EIRPs_dBm.csv', index=True)
EIRPs_dBm

,EIRP (dBm),Percentage (%)
69.041202,1729,53.20
69.341200,637,19.60
71.141204,569,17.51
71.041199,251,7.72
69.341448,59,1.82
66.330895,3,0.09
71.041114,2,0.06


In [26]:
# EIRP dBm/MHz
EIRPs_dBm = pd.DataFrame(nbn['EIRP_dBm/MHz'].value_counts( sort=True))
EIRPs_dBm['Percentage (%)'] = ((EIRPs_dBm['EIRP_dBm/MHz'] /  3250) * 100).round(2)
EIRPs_dBm.rename(columns={'EIRP_dBm/MHz' : 'EIRP (dBm/MHz)'}, inplace=True)
EIRPs_dBm.to_csv('Data/NBN/EIRPs_dBmHz.csv', index=True)
EIRPs_dBm

,EIRP (dBm/MHz),Percentage (%)
56.030902,1729,53.20
56.352669,637,19.60
58.130904,569,17.51
58.052668,251,7.72
56.352917,59,1.82
53.342365,3,0.09
58.052583,2,0.06


In [27]:
# Antenna Height Distributions
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
AntennaHts = pd.DataFrame(nbn['Antenna_Ht'].value_counts(bins=bins, sort=False))
AntennaHts['Percentage (%)'] = ((AntennaHts['Antenna_Ht'] /  3250) * 100).round(2)
AntennaHts.rename(columns={'Antenna_Ht' : 'Deployment Height (M)'}, inplace=True)
AntennaHts.to_csv('Data/NBN/AntennaHts.csv', index=True)
AntennaHts


,Deployment Height (M),Percentage (%)
"(-0.001, 10.0]",4,0.12
"(10.0, 20.0]",89,2.74
"(20.0, 30.0]",592,18.22
"(30.0, 40.0]",1698,52.25
"(40.0, 50.0]",766,23.57
"(50.0, 60.0]",89,2.74
"(60.0, 70.0]",12,0.37
"(70.0, 80.0]",0,0.00
"(80.0, 90.0]",0,0.00
"(90.0, 100.0]",0,0.00
